In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("CUDA is available. Using GPU.")
else:
  device = torch.device("cpu")
  print("CUDA is not available. Using CPU.")

use_gpu = True

CUDA is available. Using GPU.


In [26]:
mnist_train = pd.read_csv('../data/mnist/mnist_train.csv', header=None)
mnist_test = pd.read_csv('../data/mnist/mnist_test.csv', header=None)

mnist_train_x = mnist_train.iloc[:, 1:].values / 255.0
mnist_train_y = mnist_train.iloc[:, 0].values 
mnist_test_x = mnist_test.iloc[:, 1:].values / 255.0
mnist_test_y = mnist_test.iloc[:, 0].values


if use_gpu:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32).to(device)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long).to(device)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32).to(device)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long).to(device)
else:
  mnist_train_x = torch.tensor(mnist_train_x, dtype=torch.float32)
  mnist_train_y = torch.tensor(mnist_train_y, dtype=torch.long)
  mnist_test_x = torch.tensor(mnist_test_x, dtype=torch.float32)
  mnist_test_y = torch.tensor(mnist_test_y, dtype=torch.long)

mnist_train_x.is_cuda

True

In [27]:
class MNIST_Net(nn.Module):
  def __init__(self):
    super(MNIST_Net, self).__init__()
    self.fc1 = nn.Linear(784, 500)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(500, 10)
    
  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

In [28]:
train_dataset = TensorDataset(mnist_train_x, mnist_train_y)
test_dataset = TensorDataset(mnist_test_x, mnist_test_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [29]:
if use_gpu:
  model = MNIST_Net().to(device)
else:
    model = MNIST_Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
            
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)
        
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
        
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)
        
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("=========================================")
    print()

train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.2639, Accuracy: 92.44%
Test Loss: 0.1312, Test Accuracy: 96.01%

Epoch [2/5], Loss: 0.1013, Accuracy: 96.98%
Test Loss: 0.0831, Test Accuracy: 97.42%

Epoch [3/5], Loss: 0.0664, Accuracy: 98.02%
Test Loss: 0.0700, Test Accuracy: 97.78%

Epoch [4/5], Loss: 0.0457, Accuracy: 98.66%
Test Loss: 0.0744, Test Accuracy: 97.75%

Epoch [5/5], Loss: 0.0336, Accuracy: 98.96%
Test Loss: 0.0690, Test Accuracy: 97.91%



In [24]:
if use_gpu:
  model = MNIST_Net().to(device)
else:
    model = MNIST_Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5):
  for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
            
      _, predicted = torch.max(outputs, 1)
      correct += (predicted == labels).sum().item()
      total += labels.size(0)
        
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")
        
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for images, labels in test_loader:
          outputs = model(images)
          loss = criterion(outputs, labels)
          test_loss += loss.item()
          _, predicted = torch.max(outputs, 1)
          correct += (predicted == labels).sum().item()
          total += labels.size(0)
        
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%")
    print("=========================================")
    print()

train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5)

Epoch [1/5], Loss: 0.2605, Accuracy: 92.60%
Test Loss: 0.1213, Test Accuracy: 96.39%

Epoch [2/5], Loss: 0.1004, Accuracy: 96.96%
Test Loss: 0.0873, Test Accuracy: 97.26%

Epoch [3/5], Loss: 0.0641, Accuracy: 98.06%
Test Loss: 0.0768, Test Accuracy: 97.64%

Epoch [4/5], Loss: 0.0448, Accuracy: 98.65%
Test Loss: 0.0725, Test Accuracy: 97.78%

Epoch [5/5], Loss: 0.0332, Accuracy: 98.97%
Test Loss: 0.0649, Test Accuracy: 97.94%

